# SIRX: Table 1
Baseline comparion in terms of total loss and energy.

To run this script:
1. Please make sure that the required data folder is available at the paths used by the script.
You may generate the required data by running the python script
```nodec_experiments/sirx/gen_parameters.py```.

2. The plots use the training results.
Please also make sure that a training proceedures for both RL and NODEC have produced results in the corresponding paths used in plot and table scripts.
Running ```nodec_experiments/sirx/nodec_train.ipynb``` and ```nodec_experiments/sirx/nodec_train.ipynb```with default paths is expected to generate at the requiered location for the plots and table scripts in each folder.

3. Sample evaluation is done across alla baseliens before running the plots that also require the following script to run:
`nodec_experiments/sirx/eval_baselines.ipynb`

4. Extra scripts on experiments that did not produce good results may not be provide for the sake of space and brevity.

5. The scripts below:
 - ```nodec_experiments/sirx/sirx.py```
 - ```nodec_experiments/sirx/rl_utils.py```
 - ```nodec_experiments/sirx/sirx_utils.py```
contain very important utilities for running training , evaluation and plotting scripts. Please make sure that they are available in the python path when running experiments.

Reinforcement Learning requires some significant time to train.

As neural network intialization is stochastic, please make sure that appropriate seeds are used or expect some variance to paper results.

## Imports


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.append("../../") # append modules from parent dir
sys.path.append("./sirx_utils.py")
import numpy as np
import torch
import pandas as pd

import networkx as nx

from sirx_utils import comparison, sirx_curves, stack_plot_grid, heats_for_steps

from plotly import graph_objects as go
from plotly import figure_factory as ff
import plotly.express as px
import random

from tqdm.auto import tqdm

import copy



from nnc.controllers.neural_network.nnc_controllers import NNCDynamics
from nnc.helpers.torch_utils.graphs import drivers_to_tensor

In [3]:
device = 'cpu'
dtype = torch.float

### Graph parameters

In [4]:
graph = 'lattice'
parameters_folder = '../data/parameters/sirx/'
results_folder = '../data/results/sirx/'+graph+'/'
evaluation_results_folder = results_folder + 'eval/'

graph_parameters_folder = parameters_folder + '/' + 'lattice' + '/'

adjacency_matrix = torch.load(graph_parameters_folder + 'adjacency.pt', map_location=device).to(dtype)
n_nodes = adjacency_matrix.shape[-1]
side_size = int(np.sqrt(n_nodes))

### Dynamics Parameters

In [5]:
initial_infection_nodes = torch.load(graph_parameters_folder + 'initial_infection_nodes.pt')
target_subgraph = torch.load(graph_parameters_folder + 'target_subgraph_nodes.pt')
total_time = 5

def peak_infection(states):
    return states[:, target_subgraph].mean(-1).max()

def total_energy(control_signals):
    interaction_interval = total_time/control_signals.shape[0]
    total_energy = ((control_signals**2)*interaction_interval).sum()
    return total_energy


In [6]:
all_unn = np.load(evaluation_results_folder + "nodec_control_signal.npy")
all_ucc = np.load(evaluation_results_folder + 'constant_control_signal.npy')
all_urn = np.load(evaluation_results_folder + 'random_control_signal.npy')
all_url = np.load(evaluation_results_folder + 'td3_control_signal.npy')

In [7]:
all_xnc = np.load(evaluation_results_folder + "no_control_states.npy")
all_xnn = np.load(evaluation_results_folder + "nodec_states.npy")
all_xcc = np.load(evaluation_results_folder + "constant_control_states.npy")
all_xrn = np.load(evaluation_results_folder + "random_control_states.npy")
all_xrl = np.load(evaluation_results_folder + "td3_control_states.npy")


In [8]:
all_data = []
all_data.append({'Control': 'TCC', 
                 r"$\max_t\left(\bar{I}(t)\right)$" : peak_infection(all_xcc),
                 r"$\mathcal{E}$" :  total_energy(all_ucc)
                })
all_data.append({'Control': 'NODEC', 
                 r"$\max_t\left(\bar{I}(t)\right)$" : peak_infection(all_xnn),
                 r"$\mathcal{E}$" :  total_energy(all_unn)
                })
all_data.append({'Control': 'RL', 
                 r"$\max_t\left(\bar{I}(t)\right)$" : peak_infection(all_xrl),
                 r"$\mathcal{E}$" :  total_energy(all_url)
                })
all_data.append({'Control': 'RND', 
                 r"$\max_t\left(\bar{I}(t)\right)$" : peak_infection(all_xrn),
                 r"$\mathcal{E}$" :  total_energy(all_urn)
                })
all_data.append({'Control': 'F', 
                 r"$\max_t\left(\bar{I}(t)\right)$" : peak_infection(all_xnc),
                 r"$\mathcal{E}$" :  0.0
                })

In [9]:
df = pd.DataFrame(all_data)
df.iloc[:, 1] = df.iloc[:, 1].round(3)
df.iloc[:, 2] = df.iloc[:, 2].round(1)

## Table 1

In [10]:
df

,Control,$\max_t\left(\bar{I}(t)\right)$,$\mathcal{E}$
0,TCC,0.068,14062.6
1,NODEC,0.078,8356.6
2,RL,0.099,6358.0
3,RND,0.210,4688.9
4,F,0.532,0.0


In [11]:
print(df.to_latex(index=False))

\begin{tabular}{lrr}
\toprule
Control &  \$\textbackslash max\_t\textbackslash left(\textbackslash bar\{I\}(t)\textbackslash right)\$ &  \$\textbackslash mathcal\{E\}\$ \\
\midrule
    TCC &                            0.068 &        14062.6 \\
  NODEC &                            0.078 &         8356.6 \\
     RL &                            0.099 &         6358.0 \\
    RND &                            0.210 &         4688.9 \\
      F &                            0.532 &            0.0 \\
\bottomrule
\end{tabular}



In [12]:
df = df.set_index('Control')

In [13]:
# NODEC vs TCC loss
(df.loc['NODEC'].iloc[0] - df.loc['TCC'].iloc[0])

0.009999999999999995

In [14]:
# NODEC vs TCC loss
(df.loc['NODEC'].iloc[1] - df.loc['TCC'].iloc[1])/df.loc['TCC'].iloc[1]

-0.40575711461607383

In [15]:
(df.loc['RL'].iloc[1] - df.loc['TCC'].iloc[1])/df.loc['TCC'].iloc[1]

-0.5478787706398532

In [16]:
(df.loc['RL'].iloc[0] - df.loc['NODEC'].iloc[0])

0.021000000000000005